Customer Retention Analysis

Step :1

   Merging the data of the dataset to reduce the complexity
   

In [13]:
import pandas as pd

# Load core datasets
orders = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_orders_dataset.csv")
customers = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_customers_dataset.csv")
order_items = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_order_items_dataset.csv")
payments = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_order_payments_dataset.csv")
reviews = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_order_reviews_dataset.csv")

# Merge orders with customers
merged = pd.merge(orders, customers, on='customer_id', how='left')

# Merge with order items
merged = pd.merge(merged, order_items, on='order_id', how='left')

# Merge with payments
merged = pd.merge(merged, payments, on='order_id', how='left')

# Merge with reviews
merged = pd.merge(merged, reviews[['order_id', 'review_score']], on='order_id', how='left')


Saving the merged data into a Parquet file


In [14]:

# Save merged data
merged.to_parquet(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\processed\merged_data.parquet", index=False)


In [15]:
# Load product data and translation file
products = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\olist_products_dataset.csv")
translation = pd.read_csv(r"E:\OneDrive\Desktop\Customer Retention Analysis\data\raw-data\product_category_name_translation.csv")

# Translate category names
products = pd.merge(products, translation, on='product_category_name', how='left')

# Merge with main dataset
merged = pd.merge(merged, products[['product_id', 'product_category_name_english']], on='product_id', how='left')

Step : 2 

Data Cleaning and Preprocessing

In [16]:
# Drop orders with missing critical fields (customer_id, order_id)
merged.dropna(subset=['customer_id', 'order_id'], inplace=True)

# Fill missing review scores with median - Fix the chained assignment
# Instead of:
# merged['review_score'].fillna(merged['review_score'].median(), inplace=True)
merged['review_score'] = merged['review_score'].fillna(merged['review_score'].median())

# Convert date columns to datetime
merged['order_purchase_timestamp'] = pd.to_datetime(merged['order_purchase_timestamp'])

Keeping only the valid order and dropping the other for better analysis

In [17]:
# Remove canceled/unavailable orders
valid_statuses = ['delivered', 'shipped', 'invoiced']
merged = merged[merged['order_status'].isin(valid_statuses)]

Step : 3 

Feature Engineering

In [18]:
merged['monetary_value'] = merged['price'] + merged['freight_value']

In [19]:
# Group by customer_id
customer_data = merged.groupby('customer_id').agg(
    first_purchase_date=('order_purchase_timestamp', 'min'),
    last_purchase_date=('order_purchase_timestamp', 'max'),
    total_orders=('order_id', 'nunique'),
    total_spent=('monetary_value', 'sum'),
    avg_review_score=('review_score', 'mean')
).reset_index()